In [1]:
import torch
import torch.nn as nn
from sklearn import datasets
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [2]:
dataset = pd.read_csv('HeartDisease.csv')
dataset = np.array(dataset)
np.random.shuffle(dataset)
dataset_train = dataset[: 46000, :]
dataset_test = dataset[46000:, :]
class HeartDataset(Dataset):
  def __init__(self, indata):
    dataset = indata
    dataset = np.array(dataset)
    self.x = dataset[:, 1:]
    self.y = dataset[:, 0]
    self.n_samples = self.x.shape[0]
    self.n_feature = self.x.shape[1]

  def __getitem__(self, index):
    return self.x[index], self.y[index]

  def __len__(self):
    return self.n_samples

In [3]:
sc = StandardScaler()
data_train = HeartDataset(dataset_train)
X_test = torch.tensor(dataset_test[:, 1:], dtype = float)
Y_test = torch.tensor(dataset_test[:, 0], dtype = float)
print(X_test)
dataloader = DataLoader(dataset=data_train, batch_size=32, shuffle=True)

tensor([[ 4.,  1.,  0.,  ..., 30., 16.,  4.],
        [ 3.,  1.,  0.,  ..., 30., 30.,  8.],
        [ 0.,  0.,  0.,  ...,  3.,  2., 30.],
        ...,
        [ 3.,  1.,  0.,  ..., 24., 12.,  1.],
        [ 2.,  1.,  0.,  ..., 60.,  8.,  0.],
        [ 2.,  1.,  0.,  ..., 30.,  0.,  1.]], dtype=torch.float64)


In [9]:
class LogisticRegression(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(LogisticRegression, self).__init__()
    self.linear1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.linear2 = nn.Linear(hidden_size, output_size)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    out = self.linear1(x.float())
    out = self.linear2(out)
    y_pred = self.sigmoid(out)
    return y_pred


In [10]:
n_feature = data_train.n_feature
n_samples = data_train.n_samples
model = LogisticRegression(n_feature, 6, 1)
loss = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
epochs = 30
overall_loss = 0
count = 0
lambda_reg = 0.01
for epoch in range(epochs):
  for input, label in dataloader:
    y_pred = model(input)
    label = label.reshape(-1, 1).float()
    l = loss(y_pred, label)
    l2_norm = sum(p.pow(2).sum() for p in model.parameters())
    l += lambda_reg * l2_norm
    overall_loss += l.item()
    count += 1
    l.backward()
    optimizer.step()
    optimizer.zero_grad()
  print(f"Epoch {epoch} Loss {overall_loss/count}")


Epoch 0 Loss 0.335445601599335
Epoch 1 Loss 0.30587362929639794
Epoch 2 Loss 0.2943080664682731
Epoch 3 Loss 0.2878744079386854
Epoch 4 Loss 0.28414679906215723
Epoch 5 Loss 0.2814736295712522
Epoch 6 Loss 0.27939213180724665
Epoch 7 Loss 0.2778643856402448
Epoch 8 Loss 0.2767399167780163
Epoch 9 Loss 0.27567896269176345
Epoch 10 Loss 0.274888747699798
Epoch 11 Loss 0.2740492192549285
Epoch 12 Loss 0.2735199217430588
Epoch 13 Loss 0.2729621518934551
Epoch 14 Loss 0.2725854036160328
Epoch 15 Loss 0.2721668275149108
Epoch 16 Loss 0.2717869092245455
Epoch 17 Loss 0.2714655748765649
Epoch 18 Loss 0.2711218022133608
Epoch 19 Loss 0.27091297738646547
Epoch 20 Loss 0.27065773831770595
Epoch 21 Loss 0.2703867822228795
Epoch 22 Loss 0.2701550206216117
Epoch 23 Loss 0.26998543930338204
Epoch 24 Loss 0.2697611951151337
Epoch 25 Loss 0.26959301039652855
Epoch 26 Loss 0.26944337446340055
Epoch 27 Loss 0.2692850016227453
Epoch 28 Loss 0.2691737716453137
Epoch 29 Loss 0.26902705785772196


In [11]:
#Model testing
predictions = model(X_test).detach().numpy()
predictions = predictions.reshape(-1).round()
Y_train = Y_test.detach().numpy()
result = predictions == Y_train
print(f"Model Accuracy {int(np.sum(result)/len(result)*100)}%")

Model Accuracy 92%
